In [1]:
import os
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
print("Torch:", torch.__version__)
print("TorchVision:", torchvision.__version__)
print("NumPy:", numpy.__version__)

Torch: 2.8.0+cu128
TorchVision: 0.23.0+cu128
NumPy: 2.2.6


In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
# This is to set up the NeurolNetwork Class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [15]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted Class: {y_pred}")

Predicted Class: tensor([3], device='cuda:0')


In [20]:
input_image= torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [25]:
layer1= nn.Linear(in_features=28*28, out_features=20)
hidden1= layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [27]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.5372, 0.0839, 0.1051, 0.2811, 0.0000, 0.0000, 0.0000, 0.0000, 0.1193,
         0.0760, 0.3476, 0.0000, 0.1146, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0230, 0.0936, 0.2448, 0.0000, 0.3249, 0.0029, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2137, 0.3542, 0.1750, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2505, 0.1141, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0954,
         0.0000, 0.4852, 0.4189, 0.0372, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.5372, 0.0839, 0.1051, 0.2811, 0.0000, 0.0000, 0.0000, 0.0000, 0.1193,
         0.0760, 0.3476, 0.0000, 0.1146, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0230, 0.0936, 0.2448, 0.0000, 0.3249, 0.0029, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2137, 0.3542, 0.1750, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
       

In [29]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits= seq_modules(input_image)

In [31]:
softmax = nn.Softmax(dim=1)
pred_prob = softmax(logits)

In [33]:
print(f"Model Structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0056,  0.0012,  0.0145,  ..., -0.0148,  0.0249, -0.0202],
        [-0.0205,  0.0031,  0.0024,  ..., -0.0018,  0.0162, -0.0012]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0030, -0.0312], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0162, -0.0254,  0.0083,  ...,  0.0063, -0.0199,  0.0334],
        [-0.0298,  0.0429, -0.0157,  ..., -0.0177,  0.0012, -0.0325]],
       device='cuda:0', grad_fn=<Sl